In [37]:
import torch

### create tensor data

In [38]:
x = torch.randn(5, 3)
y = torch.ones_like(x)
# tensor = torch.ones(5, 3)

In [39]:
x,y

(tensor([[ 0.6800, -0.4084, -0.2253],
         [-0.7846, -2.0101,  0.6967],
         [-2.3856, -0.7948,  0.0552],
         [-0.4244, -0.7533, -0.3601],
         [ 1.5332, -1.2102, -0.0173]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]))

In [40]:
z1 = x + y
z1

tensor([[ 1.6800,  0.5916,  0.7747],
        [ 0.2154, -1.0101,  1.6967],
        [-1.3856,  0.2052,  1.0552],
        [ 0.5756,  0.2467,  0.6399],
        [ 2.5332, -0.2102,  0.9827]])

### Moving to GPU

In [41]:
x = x.cuda()
y = y.cuda()
z2 = x + y
x,y,z2

(tensor([[ 0.6800, -0.4084, -0.2253],
         [-0.7846, -2.0101,  0.6967],
         [-2.3856, -0.7948,  0.0552],
         [-0.4244, -0.7533, -0.3601],
         [ 1.5332, -1.2102, -0.0173]], device='cuda:0'),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([[ 1.6800,  0.5916,  0.7747],
         [ 0.2154, -1.0101,  1.6967],
         [-1.3856,  0.2052,  1.0552],
         [ 0.5756,  0.2467,  0.6399],
         [ 2.5332, -0.2102,  0.9827]], device='cuda:0'))

In [42]:
### 数据必须在同一设备才能操作

In [43]:
try: z1 + z2
except Exception as e: print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


In [44]:
z2 = z2.to("cpu")
print(z1 + z2)

tensor([[ 3.3599,  1.1833,  1.5495],
        [ 0.4309, -2.0202,  3.3934],
        [-2.7712,  0.4105,  2.1104],
        [ 1.1513,  0.4935,  1.2799],
        [ 5.0663, -0.4204,  1.9655]])


### Neural network

In [45]:
import torch.nn as nn
import torch.nn.functional as F

# 定义一个简单的神经网络类
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # super(Net, self).__init__()
        # 定义每一层卷积神经网络，输入通道维度=1，输出通道维度=6，卷积核大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度=6，输出通道维度=16，卷积核大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)   #6×6×16
        # 定义三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在(2,2)的池化窗口下执行最大池化操作
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) #
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 计算size,除了第0个 维度上的batch_size
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

1. 第一层卷积：
   - 输入大小：32×32
   - 卷积核大小：3×3
   - 输出通道维度：6
   - 使用零填充（padding=0），步幅（stride=1）
   - 计算公式：$output\_size = \frac{{input\_size - kernel\_size + 2 \times padding}}{{stride}} + 1 = \frac{{32 - 3 + 2 \times 0}}{{1}} + 1 = 30$
   - 输出大小：30×30

2. 第一次最大池化：
   - 输入大小：30×30
   - 池化窗口大小：2×2
   - 使用步幅（stride=2）
   - 计算公式：$output\_size = \frac{{input\_size}}{{stride}} = \frac{{30}}{{2}} = 15$
   - 输出大小：15×15

3. 第二层卷积：
   - 输入大小：15×15
   - 卷积核大小：3×3
   - 输出通道维度：16
   - 使用零填充（padding=0），步幅（stride=1）
   - 计算公式：$output\_size = \frac{{input\_size - kernel\_size + 2 \times padding}}{{stride}} + 1 = \frac{{15 - 3 + 2 \times 0}}{{1}} + 1 = 13$
   - 输出大小：13×13

4. 第二次最大池化：
   - 输入大小：13×13
   - 池化窗口大小：2×2
   - 使用步幅（stride=2）
   - 计算公式：$output\_size = \frac{{input\_size}}{{stride}} = \frac{{13}}{{2}} = 6.5$（结果向下取整）
   - 输出大小：6×6

最后，在卷积和池化的过程中，图像的大小从32×32逐步减小到6×6。在这之后，将数据展平，并通过全连接层进行处理。

![16970320504401697032050389.png](https://fastly.jsdelivr.net/gh/Chenjiangwen/ImageHostingService@main/pic/16970320504401697032050389.png)

In [46]:
# import matplotlib.pyplot as plt
# 
# img = plt.imread('img.png')
# 
# fig = plt.figure(figsize=(20, 15))
# plt.imshow(img)
# plt.axis('on')
# plt.show()


In [47]:
net = Net()
net.to("cuda")

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [48]:
for name, param in net.named_parameters():
    print('{: <30} {: <30} {: <20}'.format(name, str(param.shape), str(param.numel())))
print("总参数量： {}".format(sum(x.numel() for x in net.parameters())))

conv1.weight                   torch.Size([6, 1, 3, 3])       54                  
conv1.bias                     torch.Size([6])                6                   
conv2.weight                   torch.Size([16, 6, 3, 3])      864                 
conv2.bias                     torch.Size([16])               16                  
fc1.weight                     torch.Size([120, 576])         69120               
fc1.bias                       torch.Size([120])              120                 
fc2.weight                     torch.Size([84, 120])          10080               
fc2.bias                       torch.Size([84])               84                  
fc3.weight                     torch.Size([10, 84])           840                 
fc3.bias                       torch.Size([10])               10                  
总参数量： 81194


### 输入、输出

In [49]:
input = torch.randn(1, 1, 32, 32).to("cuda")
print('input', input.size(), input.type())
out = net(input)
print(out)


input torch.Size([1, 1, 32, 32]) torch.cuda.FloatTensor
tensor([[-0.0338,  0.1494, -0.0319, -0.0819, -0.1217, -0.0548, -0.0139, -0.0012,
          0.1079, -0.1118]], device='cuda:0', grad_fn=<AddmmBackward0>)


### 损失

In [50]:
target = torch.randn(10).cuda()
print(target.shape)
target = target.view(1,-1)
print(target.size())
criterion = nn.MSELoss()
loss = criterion(out, target)
print(loss)

torch.Size([10])
torch.Size([1, 10])
tensor(0.5652, device='cuda:0', grad_fn=<MseLossBackward0>)


### Input -> Conv1 -> ReLu -> MaxPool -> Conv2 -> ReLu -> MaxPool
###      -> view -> FC_linear -> Relu -> FC_linear -> ReLu -> FC_linear
###      -> MSELoss
###      ->loss

In [51]:
net.zero_grad()  #梯度清零，否则不同批次数据之间的梯度会累加
print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

None
tensor([-0.0113, -0.0014,  0.0062, -0.0091,  0.0004,  0.0037], device='cuda:0')


In [52]:
### 反向传播 loss.backward()

In [53]:
# 跟踪loss反向传播
print(loss.grad_fn)  #MSELoss
print(loss.grad_fn.next_functions[0][0])  #linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])   #ReLu

### 优化器 更新网络参数

In [54]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

loss.backward()

# 更新参数
optimizer.step()

print('总结：1.构建网络 2.定义损失函数 3.反向传播 4.更新参数\n'
      '循环训练直到满意loss，结束训练 -> 模型 -> do something')

总结：1.构建网络 2.定义损失函数 3.反向传播 4.更新参数
循环训练直到满意loss，结束训练 -> 模型 -> do something


## cifar-10

| 类别  | plane | car | bird | cat | deer | dog | frog | horse | ship | truck |
| ----- | ----- | --- | ---- | --- | ---- | --- | ---- | ----- | ---- | ----- |
| 准确率 | 69%   | 69% | 27%  | 31% | 42%  | 64% | 59%  | 65%   | 59%  | 62%   |

https://cjwen-imageclassification-resnet18-cifar10.streamlit.app/

![1697088964579GIF 2023-10-12 13-34-26.gif](https://github.com/Chenjiangwen/ImageHostingService/blob/main/pic/1697088964579GIF 2023-10-12 13-34-26.gif)

![1697088964579GIF 2023-10-12 13-34-26.gif](https://fastly.jsdelivr.net/gh/Chenjiangwen/ImageHostingService@main/pic/1697088964579GIF 2023-10-12 13-34-26.gif)

![1697089504021GIF2023-10-12-13-34-26.gif](https://fastly.jsdelivr.net/gh/Chenjiangwen/ImageHostingService@main/pic/1697089504021GIF2023-10-12-13-34-26.gif)